In [ ]:
from postbound.db import db, mysql
from postbound.qal import qal, base, clauses, transform, formatter, predicates, expressions
from postbound.optimizer import jointree
from postbound.optimizer.physops import operators as physops
from postbound.optimizer.planmeta import hints as planmeta
conn_string = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "1234",
    "database": "imdbload"}
mysql_db = mysql.connect(connection_args=mysql.MysqlConnectionArguments(**conn_string))
hint_service = mysql_db.hinting()
table1 = base.TableReference(full_name="movie_info", alias="mi")
table2 = base.TableReference("keyword", alias="ky")
table3 = base.TableReference("movie_companies", alias="mc")
table4 = base.TableReference("movie_keyword", alias="mk")
table5 = base.TableReference("movie_link", alias="ml")
table = [table1,table2,table3,table4,table5]
assignment  = physops.PhysicalOperatorAssignment()
plan = planmeta.PlanParameterization()
plan_parameter2 = plan.add_join_order_hint(tables=table)
join_operator1 = physops.JoinOperatorAssignment(join=[table3,table2], operator=physops.JoinOperators.HashJoin)
scan_operator = physops.ScanOperatorAssignment(
    operator=physops.ScanOperators.IndexScan,
    table=table1
)
join_operator2 = physops.JoinOperatorAssignment(
    operator=physops.JoinOperators.IndexMergeJoin,
    join=[table4, table5]
)
join_operators = {
    frozenset(join_operator1.join): join_operator1,
    frozenset(join_operator2.join): join_operator2,
}
for join, join_operator in join_operators.items():
    assignment.set_join_operator(join_operator)
# Set the scan operator in the PhysicalOperatorAssignment
assignment.set_scan_operator(scan_operator)
assignment.set_operator_enabled_globally(operator=physops.JoinOperators.BlockNestedLoopJoin, enabled=False)
tree1 = jointree.LogicalJoinTree.for_base_table(table=base.TableReference(full_name="movie_info", alias="mi"),base_annotation=jointree.LogicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_info", alias="mi")),second_argument=42),cardinality=1000))
tree2 = jointree.LogicalJoinTree.for_base_table(table= base.TableReference("keyword", alias="ky"),base_annotation=jointree.LogicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")),second_argument=100),cardinality=100))
tree3 = jointree.LogicalJoinTree.for_base_table(table=base.TableReference("movie_companies", alias="mc"),base_annotation=jointree.LogicalBaseTableMetadata(filter_predicate=predicates.BinaryPredicate(
    operation=expressions.LogicalSqlOperators.Equal,
    first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_companies", alias="mc")),second_argument=1040),cardinality=4000))
trees = [tree1,tree2,tree3]
join_sequence = [table1,table2]
nested_sequence2 = [join_sequence, table1, table2]
join_tree = jointree.LogicalJoinTree.load_from_list(join_order=nested_sequence2)
explain_clause = clauses.Explain.explain_analyze()
select_clause = clauses.Select(targets=[clauses.BaseProjection.column(base.ColumnReference(name="linked_movie_id",table=base.TableReference(full_name="movie_link", alias="ml")))])
#select_clause = clauses.Select(targets=[clauses.BaseProjection.count_star()])
from_clause = clauses.ImplicitFromClause(tables=table)
predicate1 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_info", alias="mi")),second_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")))
predicate2 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="keyword", alias="ky")),second_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_companies", alias="mc")))
predicate3 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_companies", alias="mc")),second_argument=base.ColumnReference(name="movie_id",table=base.TableReference(full_name="movie_keyword", alias="mk")))
predicate4 = predicates.BinaryPredicate(operation=expressions.LogicalSqlOperators.Equal,first_argument=base.ColumnReference(name="id",table=base.TableReference(full_name="movie_keyword", alias="mk")),second_argument=base.ColumnReference(name="movie_id",table=base.TableReference(full_name="movie_link", alias="ml")))
operation = expressions.LogicalSqlCompoundOperators.And
#where_clause = clauses.Where(predicates.BinaryPredicate(operation=operation,first_argument=predicate1,second_argument=predicate2) )
p1 = predicates.BinaryPredicate(operation=operation,first_argument=predicate1,second_argument=predicate2) 
p2=predicates.BinaryPredicate(operation=operation,first_argument=predicate2,second_argument=predicate3)
p3 = predicates.BinaryPredicate(operation=operation,first_argument=predicate3,second_argument=predicate4)
combined_predicate = predicates.BinaryPredicate(
    operation=expressions.LogicalSqlCompoundOperators.And,
    first_argument=p2,
    second_argument=predicates.BinaryPredicate(
        operation=expressions.LogicalSqlCompoundOperators.And,
        first_argument=p3,
        second_argument=p1))
where_clause = clauses.Where(predicate=combined_predicate)
limit_clause = clauses.Limit(limit=10)
query = qal.ImplicitSqlQuery(explain_clause=explain_clause,select_clause=select_clause,from_clause=from_clause)
results = hint_service.generate_hints(query=query)
print(results)
final = mysql_db.execute_query(query=results)
print(final)
